In [4]:
import os
import pandas as pd

In [5]:
DATA_DIR = "../data"

In [6]:
players_df = pd.read_csv(os.path.join(DATA_DIR, "player_features_with_score.csv"))
deliveries_df = pd.read_csv(os.path.join(DATA_DIR, "deliveries.csv"))
matches_df = pd.read_csv(os.path.join(DATA_DIR, "matches.csv"))

players_df.shape, deliveries_df.shape, matches_df.shape

((732, 11), (260920, 17), (1095, 20))

In [7]:
available_seasons = sorted(matches_df['season'].dropna().unique())
available_seasons

['2007/08',
 '2009',
 '2009/10',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020/21',
 '2021',
 '2022',
 '2023',
 '2024']

In [8]:
SELECTED_SEASON = available_seasons[-1]  # latest season
SELECTED_SEASON

'2024'

In [9]:
season_matches = matches_df[matches_df['season'] == SELECTED_SEASON]
season_match_ids = season_matches['id'].unique()

len(season_match_ids)

71

In [10]:
season_deliveries = deliveries_df[
    deliveries_df['match_id'].isin(season_match_ids)
]

season_deliveries.shape

(17103, 17)

In [23]:
player_team_map = (
    season_deliveries[['batter', 'batting_team']]
    .drop_duplicates()
    .rename(columns={
        'batter': 'player',
        'batting_team': 'team'
    })
)

sorted(player_team_map['team'].unique())

['Chennai Super Kings',
 'Delhi Capitals',
 'Gujarat Titans',
 'Kolkata Knight Riders',
 'Lucknow Super Giants',
 'Mumbai Indians',
 'Punjab Kings',
 'Rajasthan Royals',
 'Royal Challengers Bengaluru',
 'Sunrisers Hyderabad']

In [24]:
players_with_team = players_df.merge(
    player_team_map,
    on='player',
    how='inner'
)

players_with_team[['player', 'team']].head()

,player,team
0,A Badoni,Lucknow Super Giants
1,A Kamboj,Mumbai Indians
2,A Manohar,Gujarat Titans
3,A Nortje,Delhi Capitals
4,A Raghuvanshi,Kolkata Knight Riders


In [25]:
TEAM_A = "Royal Challengers Bengaluru"
TEAM_B = "Mumbai Indians"

In [ ]:
eligible_players = players_with_team[
    (players_with_team['team'] == TEAM_A) |
    (players_with_team['team'] == TEAM_B)
].copy()

eligible_players['team'].value_counts()

team
Royal Challengers Bengaluru    19
Mumbai Indians                 18
Name: count, dtype: int64

In [18]:
eligible_players['final_score'] = eligible_players['performance_score']

In [19]:
suggested_players = eligible_players.sort_values(
    'final_score',
    ascending=False
)

suggested_players_reset = suggested_players.reset_index(drop=True)

suggested_players_reset[['player', 'team', 'final_score']].head(22)

,player,team,final_score
0,V Kohli,Royal Challengers Bengaluru,0.465484
1,RG Sharma,Mumbai Indians,0.416482
2,PP Chawla,Mumbai Indians,0.348273
3,KD Karthik,Royal Challengers Bengaluru,0.346861
4,HH Pandya,Mumbai Indians,0.308980
5,GJ Maxwell,Royal Challengers Bengaluru,0.299672
6,F du Plessis,Royal Challengers Bengaluru,0.292691
7,JJ Bumrah,Mumbai Indians,0.281491
8,SA Yadav,Mumbai Indians,0.280607
9,L Wood,Mumbai Indians,0.260339


In [20]:
user_selected_indices = [0,1,2,3,4,5,6,7,8,9,10]

In [21]:
final_xi = suggested_players_reset.loc[user_selected_indices]

final_xi[['player', 'team', 'final_score']]

,player,team,final_score
0,V Kohli,Royal Challengers Bengaluru,0.465484
1,RG Sharma,Mumbai Indians,0.416482
2,PP Chawla,Mumbai Indians,0.348273
3,KD Karthik,Royal Challengers Bengaluru,0.346861
4,HH Pandya,Mumbai Indians,0.308980
5,GJ Maxwell,Royal Challengers Bengaluru,0.299672
6,F du Plessis,Royal Challengers Bengaluru,0.292691
7,JJ Bumrah,Mumbai Indians,0.281491
8,SA Yadav,Mumbai Indians,0.280607
9,L Wood,Mumbai Indians,0.260339


In [26]:
team_a_xi = final_xi[final_xi['team'] == TEAM_A]
team_b_xi = final_xi[final_xi['team'] == TEAM_B]

team_a_xi.shape, team_b_xi.shape

((4, 13), (7, 13))

In [27]:
team_a_strength = team_a_xi['final_score'].sum()
team_b_strength = team_b_xi['final_score'].sum()

team_a_strength, team_b_strength

(1.4047090962394415, 2.146197635406461)

In [28]:
total_strength = team_a_strength + team_b_strength

team_a_win_prob = team_a_strength / total_strength
team_b_win_prob = team_b_strength / total_strength

if team_a_win_prob > team_b_win_prob:
    predicted_winner = TEAM_A
else:
    predicted_winner = TEAM_B

predicted_winner, team_a_win_prob, team_b_win_prob

('Mumbai Indians', 0.39559166218605135, 0.6044083378139488)